In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Data Skewness").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/02/26 20:16:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/02/26 20:16:15 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/02/26 20:16:15 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [8]:
df1 =  spark.read.csv("DataSets/BankChurners.csv",header=True,sep=",")
df1.show(2)

+---------+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+
|CLIENTNUM|   Attrition_Flag|Customer_Age|Gender|Dependent_count|Education_Level|Marital_Status|Income_Category|Card_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Amt|Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|
+---------+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------

In [9]:
df1.count()

10127

Non uniform distribution of data across the partitions in cluster is called Data Skewness

In [31]:
df1.rdd.getNumPartitions()

1

In [23]:
partition_x = df1.repartition(5)

In [32]:
partition_x.rdd.getNumPartitions()

5

## spark_partition_id

In [36]:
from pyspark.sql.functions import spark_partition_id,col

In [38]:
df2 = partition_x.withColumn("Partition ID",spark_partition_id()).groupBy("Partition ID").count()
df2.show()


+------------+-----+
|Partition ID|count|
+------------+-----+
|           0| 2025|
|           1| 2026|
|           2| 2026|
|           3| 2025|
|           4| 2025|
+------------+-----+



## Data is uniformly distributed based on provided number of partitions
But, while doing repartition by providing any columnName, there might be non uniformly distribution across parititions

In [ ]:
partition_by_Col = df1.repartition("Col_Name")
partition_by_Col.rdd.getNumPartitions()

In [ ]:
df3 = partition_by_Col.withColumn("Partition ID",spark_partition_id()).groupBy("Partition ID").count()
df3.show()